In [1]:
import pandas as pd
# import torch
# import torch.nn.functional as F
import numpy as np
import random
import sklearn.metrics as metrics
import json
import pretty_midi
import librosa

# import transformers and matplotlib in the cell when you call them, not globally

In [2]:
json_path = '/home/hice1/sgoel83/scratch/Maestro/maestro-v3.0.0/maestro-v3.0.0.json'
data_path = '/home/hice1/sgoel83/scratch/Maestro/maestro-v3.0.0/'

In [3]:
def load_train_data(json_path):
    with open(json_path, 'r') as file:
        train_data = json.load(file)
    return train_data

# train_data = load_train_data(json_path)ces once pickle file is complete 
# indices_2018 = [index for index, year in train_data.items() if year == 2018]

# MIDI2018 and CQT2018 final dicts

In [4]:
# MIDI Pickle file

import pickle
midi_final_dict = {}
with open('/home/hice1/amardia6/scratch/midi2018.pickle', 'rb') as f:
    while True:
        try:
            data = pickle.load(f)
            midi_final_dict = {**midi_final_dict, **data}
        except EOFError:
            break

In [5]:
print(len(midi_final_dict.keys()))
print(midi_final_dict.keys())

93
dict_keys([0, 18, 19, 21, 22, 23, 26, 35, 61, 69, 74, 75, 87, 88, 89, 97, 110, 112, 114, 187, 223, 234, 255, 257, 280, 317, 347, 377, 392, 393, 413, 414, 443, 444, 445, 446, 453, 454, 455, 456, 476, 479, 672, 678, 679, 691, 711, 717, 724, 729, 778, 782, 852, 873, 888, 920, 931, 934, 936, 939, 945, 972, 1003, 1009, 1014, 1015, 1017, 1022, 1031, 1051, 1054, 1094, 1095, 1106, 1109, 1110, 1138, 1141, 1142, 1144, 1159, 1165, 1166, 1181, 1194, 1199, 1200, 1216, 1224, 1240, 1253, 1255, 1274])


In [ ]:
# CQT WAV Pickle file

import pickle
cqt_final_dict = {}
with open('/home/hice1/sgoel83/scratch/wav_pickle/88_bins/cqt2018_88.pickle', 'rb') as f:
    while True:
        try:
            cqt_data = pickle.load(f)
            cqt_final_dict = {**cqt_final_dict, **cqt_data}
        except EOFError:
            break

In [ ]:
print(len(cqt_final_dict.keys()))
print(cqt_final_dict.keys())

93
dict_keys([0, 18, 19, 21, 22, 23, 26, 35, 61, 69, 74, 75, 87, 88, 89, 97, 110, 112, 114, 187, 223, 234, 255, 257, 280, 317, 347, 377, 392, 393, 413, 414, 443, 444, 445, 446, 453, 454, 455, 456, 476, 479, 672, 678, 679, 691, 711, 717, 724, 729, 778, 782, 852, 873, 888, 920, 931, 934, 936, 939, 945, 972, 1003, 1009, 1014, 1015, 1017, 1022, 1031, 1051, 1054, 1094, 1095, 1106, 1109, 1110, 1138, 1141, 1142, 1144, 1159, 1165, 1166, 1181, 1194, 1199, 1200, 1216, 1224, 1240, 1253, 1255, 1274])


# Stripping MIDI to 88 keys

In [ ]:
# A0 is 21, C8 is 108
def strip_to_88_keys_in_place(midi_final_dict):
    for piece in midi_final_dict:
        if midi_final_dict[piece].shape[0] == 88:
            continue
        else:
            midi_final_dict[piece] = midi_final_dict[piece][21:109, :]

strip_to_88_keys_in_place(midi_final_dict=midi_final_dict)

for key, array in midi_final_dict.items():
    print(f"{key}: Shape {array.shape}")

0: Shape (88, 70398)
18: Shape (88, 70375)
19: Shape (88, 68555)
21: Shape (88, 112083)
22: Shape (88, 119524)
23: Shape (88, 111878)
26: Shape (88, 70149)
35: Shape (88, 52089)
61: Shape (88, 84826)
69: Shape (88, 34968)
74: Shape (88, 25165)
75: Shape (88, 29749)
87: Shape (88, 105461)
88: Shape (88, 102212)
89: Shape (88, 63386)
97: Shape (88, 29407)
110: Shape (88, 14140)
112: Shape (88, 19476)
114: Shape (88, 21659)
187: Shape (88, 161182)
223: Shape (88, 79511)
234: Shape (88, 170061)
255: Shape (88, 23549)
257: Shape (88, 27070)
280: Shape (88, 99133)
317: Shape (88, 56993)
347: Shape (88, 64512)
377: Shape (88, 224451)
392: Shape (88, 137190)
393: Shape (88, 184680)
413: Shape (88, 256228)
414: Shape (88, 242350)
443: Shape (88, 174353)
444: Shape (88, 174903)
445: Shape (88, 163145)
446: Shape (88, 175912)
453: Shape (88, 234858)
454: Shape (88, 208137)
455: Shape (88, 189899)
456: Shape (88, 199127)
476: Shape (88, 30586)
479: Shape (88, 20640)
672: Shape (88, 146129)
678: Sh

# Data preprocessing

In [ ]:
# Shaping parameters
batch_size = 32
slices_per_batch = 8
time_slices = 256
values_per_slice_per_batch = 431
frequency_bins_x = 88
duration_x = 10
channels_x = 1
keys_y = 88

# time per slice in seconds = 9.995 s

In [ ]:
x_parameters = {
    'batch_size': batch_size,
    'time_slices': time_slices,
    'frequency_bins': frequency_bins_x,
    'duration': duration_x,
    'channels': channels_x}

y_parameters = {
    'batch_size': batch_size,
    'time_slices': time_slices,
    'num_keys': keys_y
}

In [ ]:
# Audio CQT stats

import matplotlib.pyplot as plt
# x : (88, 32, 360, 5, 1)
# y : (88, 32, 360)

x = []
xlen = []
count = 0
count_0 = 0
max_ret = 0
min_ret = float("inf")
for idx, audio in cqt_final_dict.items():
    # print("Shape 0:" , audio.shape[0])
    # print("Shape 1:" , audio.shape[1])
    count += audio.shape[1]
    count_0 += audio.shape[0]
    # print(audio.shape[1]*audio.shape[0])
    max_ret = max(max_ret, audio.shape[1])
    min_ret = min(min_ret, audio.shape[1])
    xlen.append((idx, audio.shape[1]))
print("min_ret", min_ret)
print("max_ret", max_ret)
print(count/93)
print(count_0/93)
xlen.sort(key=lambda a: a[1])
print(xlen)
# xs = [x for x in range(len(xlen))]

# plt.plot(xs, xlen)
# plt.show()

min_ret 6114
max_ret 110392
45800.76344086022
88.0
[(110, 6114), (1200, 7172), (112, 8432), (479, 8914), (114, 9328), (255, 10181), (1216, 10527), (1109, 10740), (74, 10839), (1199, 11047), (257, 11662), (97, 12708), (1181, 12740), (75, 12843), (476, 13183), (782, 13536), (691, 14231), (69, 15103), (945, 15560), (1110, 19083), (778, 19670), (35, 22447), (939, 22793), (317, 24551), (1240, 25119), (852, 26876), (89, 27304), (711, 27466), (347, 27795), (19, 29554), (1224, 30214), (26, 30254), (0, 30331), (18, 30351), (1106, 31389), (724, 31610), (1003, 32786), (223, 34286), (1255, 34646), (920, 34925), (873, 35988), (936, 36210), (61, 36536), (934, 36894), (931, 40758), (1159, 42115), (280, 42736), (88, 44062), (1015, 44136), (87, 45462), (1017, 45732), (1274, 46071), (1253, 46182), (729, 47059), (23, 48214), (1054, 48238), (21, 48310), (1014, 50813), (22, 51518), (1031, 54052), (972, 56333), (717, 57097), (1194, 58526), (392, 59126), (1051, 62550), (672, 62972), (1138, 65593), (1165, 658

In [ ]:
# MIDI stats

import matplotlib.pyplot as plt
# x : (88, 32, 360, 5, 1)
# y : (88, 32, 360)

y = []
ylen = []
county = 0
county_0 = 0
max_ret_y = 0
min_ret_y = float("inf")
for idx, midi in midi_final_dict.items():
    # print("Shape 0:" , audio.shape[0])
    # print("Shape 1:" , audio.shape[1])
    county += midi.shape[1]
    county_0 += midi.shape[0]
    # print(audio.shape[1]*audio.shape[0])
    max_ret_y = max(max_ret_y, midi.shape[1])
    min_ret_y = min(min_ret_y, midi.shape[1])
    ylen.append(midi.shape[1])
print("min_ret", min_ret_y)
print("max_ret", max_ret_y)
print(county/93)
print(county_0/93)
# ylen = sorted(ylen)
# print(ylen)
# ys = [y for y in range(len(ylen))]

# plt.plot(ys, ylen)
# plt.show()

min_ret 14140
max_ret 256228
106282.04301075269
88.0


In [ ]:
# audio clipped duration (numbers) = 110336
# midi clipped duration (numbers) = 256000
# midi clipped duration (seconds) = 2558.8309266759
# midi time in each slice (seconds) = 2558.8309266759 / 256 = 9.9954333 s = 9995.4333 ms
# 64ms * 156 values = 9984 ms
# We CAN clip to 9984 ms if we'd like, by removing ~11 values.

# We are only clipping the end, and not adding or clipping any values in the pieces.


longest_piece_idx = xlen[-1][0]
print(cqt_final_dict[longest_piece_idx].shape)
clipped_data = np.delete(cqt_final_dict[longest_piece_idx], np.s_[110336:], axis=1)

cqt_final_dict[longest_piece_idx] = clipped_data
print(cqt_final_dict[longest_piece_idx].shape)

(88, 110392)
(88, 110336)


In [ ]:
print(midi_final_dict[longest_piece_idx].shape)
clipped_midi = np.delete(midi_final_dict[longest_piece_idx], np.s_[256000:], axis=1)

midi_final_dict[longest_piece_idx] = clipped_midi
print(midi_final_dict[longest_piece_idx].shape)

(88, 256228)
(88, 256000)


In [ ]:
# Must run with a kernel of 64GB

max_ret = 110336

audio_largest_curr_shape = (88, max_ret)

for audio_idx, audio_file in cqt_final_dict.items():
    if audio_file.shape[1] < audio_largest_curr_shape[1]:
        pad_width = audio_largest_curr_shape[1] - audio_file.shape[1]
        # print(pad_width)
        cqt_final_dict[audio_idx] = np.pad(audio_file, ((0, 0), (0, pad_width)), 'constant')

In [ ]:
# Must run with a kernel of 64GB

max_ret_y = 256000

midi_largest_curr_shape = (88, max_ret_y)

for midi_idx, midi_file in midi_final_dict.items():
    if midi_file.shape[1] < midi_largest_curr_shape[1]:
        pad_width = midi_largest_curr_shape[1] - midi_file.shape[1]
        # print(pad_width)
        midi_final_dict[midi_idx] = np.pad(midi_file, ((0, 0), (0, pad_width)), 'constant')

# Reshaping

In [ ]:
# audio_shape = (88,32,8,431)
audio_shape = (88, 32, 8, 431)

for audio_idx, audio_file in cqt_final_dict.items():
    if len(audio_file.shape) == 2:
        reshaped_audio = audio_file.reshape(audio_shape)
        print(reshaped_audio.shape)
        reshaped_audio_2 = np.moveaxis(reshaped_audio, 0, 2)
        print(reshaped_audio_2.shape)
        cqt_final_dict[audio_idx] = reshaped_audio_2

(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 431)
(32, 8, 88, 431)
(88, 32, 8, 43

In [ ]:
# midi_shape = (88, 32, 8, 1000)
midi_shape = (88, 32, 8, 1000)
final_midi_shape = (32, 8, 88, 1000)

for midi_idx, midi_file in midi_final_dict.items():
    if len(midi_file.shape) == 2:
        reshaped_midi = midi_file.reshape(midi_shape)
        print(reshaped_midi.shape)
        reshaped_midi_2 = np.moveaxis(reshaped_midi, 0, 2)
        print(reshaped_midi_2.shape)
        reshaped_midi_3 = reshaped_midi_2[:, :, :, 0]
        print(reshaped_midi_3.shape)
        midi_final_dict[midi_idx] = reshaped_midi_3

(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32, 8, 88)
(88, 32, 8, 1000)
(32, 8, 88, 1000)
(32,

# Training

In [ ]:
# Hyperparameters for the model


In [ ]:
import random

keys = list(cqt_final_dict.keys())
random.shuffle(keys)

num_keys = len(keys)
train_size = int(0.5 * num_keys)
valid_size = int(0.25 * num_keys)

train_keys = keys[:train_size]
valid_keys = keys[train_size:train_size + valid_size]
test_keys = keys[train_size + valid_size:]

In [ ]:
import tensorflow
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model

# class RecurrentCNN(nn.Module):
#     def __init__(self):
#         super(RecurrentCNN, self).__init__()
        
#         # Convolutional Layers
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(20, 2), stride=1)
#         self.pool1 = nn.MaxPool2d(kernel_size=(4, 2))
        
#         # LSTM Layers
#         self.lstm1 = nn.LSTM(input_size=32, hidden_size=500, batch_first=True, dropout=0.75)
#         self.lstm2 = nn.LSTM(input_size=500, hidden_size=200, batch_first=True)

#         # Fully Connected Layer
#         self.fc = nn.Linear(200, 1) 

#     def forward(self, x):
#         # Assuming x shape: (batch_size, 32, 360, 84, 5, 1) with CQT and MIDI dimensions as 84 and 128
#         x = x.view(-1, 1, 84, 360)

#         # Apply Convolution
#         x = self.pool1(torch.relu(self.conv1(x)))
#         # LSTM
#         x = x.view(x.size(0), -1, x.size(1))  
        
#         # LSTM forward pass
#         x, _ = self.lstm1(x)
#         x, _ = self.lstm2(x)

#         # Fully connected layer 
#         x = self.fc(x[:, -1, :])  
        
#         return torch.sigmoid(x)
        # return model

final_audio_shape = (32, 8, 88, 431, 1)
# input_shape = (final_audio_shape[1], final_audio_shape[2], final_audio_shape[3], 1)

def create_model():
    model = Sequential()

    conv = Conv2D(filters=20, kernel_size=(20, 2), strides=(1, 1), padding="same", activation="relu")
    
    model.add(TimeDistributed(conv, input_shape=final_audio_shape[1:]))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(4, 2), strides=(2, 1))))
    model.add(TimeDistributed(Conv2D(20, kernel_size=(20, 2), strides=(10, 1), activation='relu')))
#     model.add(TimeDistributed(MaxPooling2D(pool_size=(4, 2), strides=(2, 1))))
    model.add(TimeDistributed(Flatten()))
    model.add(TimeDistributed(Dense(500)))
    model.add(Bidirectional(LSTM(500, input_shape=(final_audio_shape[1], 500), return_sequences=True, recurrent_dropout=0.25)))
    model.add(Bidirectional(LSTM(200, input_shape=(final_audio_shape[1], 500), return_sequences=True)))
    model.add(TimeDistributed(Dense(88, activation = "sigmoid")))

    return model

2024-11-27 18:29:04.997078: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 18:29:05.020844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732750145.039880 1188109 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732750145.045546 1188109 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 18:29:05.064249: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
train_pieces_audio = []
valid_pieces_audio = []
for idx, audio in cqt_final_dict.items():
    if idx in train_keys:
        train_pieces_audio.append(audio)
    elif idx in valid_keys:
        valid_pieces_audio.append(audio)

In [ ]:
train_pieces_midi = []
valid_pieces_midi = []
for idx, midi in midi_final_dict.items():
    if idx in train_keys:
        train_pieces_midi.append(midi)
    elif idx in valid_keys:
        valid_pieces_midi.append(midi)

In [ ]:
import tensorflow

def data_generator(audio_batches, midi_batches):
    while True:
        # with tensorflow.device("/GPU:0"):
        for audio, midi in zip(audio_batches, midi_batches):
            t_audio = tensorflow.convert_to_tensor(audio)
            t_midi = tensorflow.convert_to_tensor(midi)
            yield t_audio, t_midi

In [ ]:
# # check GPU utilization
# if torch.cuda.is_available():
#     print(f"Number of GPUs available: {torch.cuda.device_count()}")
#     for i in range(torch.cuda.device_count()):
#         print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
# else:
#     print("No GPU available.")

In [ ]:
# import tensorflow as tf
# gpus = tf.config.list_physical_devices('GPU')
# for dev in gpus:
#     if dev.device_type == 'GPU':
#         device = tf.device(dev.name)
#         print(dev.name)
#         break
#     else:
#         device = tf.device('/physical_device:CPU:0')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

lr = 0.001
n_epochs = 20

while True:
    model = create_model()

    # with tensorflow.device("/GPU:0"):
        # model.to("GPU:0")
        # train_pieces_audio.to("GPU:0")
        # train_pieces_midi.to("GPU:0")
        # valid_pieces_audio.to("GPU:0")
        # valid_pieces_midi.to("GPU:0")
    
    print("Training from randomly initialized weights.")
    opt = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, decay=(lr / n_epochs))
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=["accuracy"])

    print(model.summary())

    # n_train_samples = 1472
    # n_valid_samples = 736

    # print("Number of training batches:", n_train_samples // batch_size)
    # print("Number of validation batches:", n_valid_samples // batch_size)

    # model.fit(x=train_pieces_audio,y=train_pieces_midi, batch_size=32, epochs=n_epochs, steps_per_epoch=n_train_samples // batch_size, validation_data=(valid_pieces_audio, valid_pieces_midi), validation_steps=n_valid_samples // batch_size, validation_batch_size=32, verbose=1)

    train_generator = data_generator(train_pieces_audio, train_pieces_midi)
    valid_generator = data_generator(valid_pieces_audio, valid_pieces_midi)

    steps_per_epoch = len(train_pieces_audio)
    validation_steps = len(valid_pieces_audio)

    model.fit(
        x=train_generator,
        epochs=n_epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=valid_generator,
        validation_steps=validation_steps,
        verbose=1
    )

    model.save('/home/hice1/sgoel83/scratch/models/original_gross0.keras')

/home/hice1/sgoel83/.venv/lib/python3.10/site-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2024-11-27 18:29:09.291098: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/hice1/sgoel83/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training from randomly initialized weights.


/home/hice1/sgoel83/.venv/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 8, 88, 431, 20) │           820 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 8, 43, 430, 20) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 8, 3, 429, 20)  │        16,020 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 8, 25740)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 8, 500)         │    12,870,500 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 8, 1000)        │     4,004,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 8, 400)         │     1,921,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 8, 88)          │        35,288 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,848,228 (71.90 MB)

 Trainable params: 18,848,228 (71.90 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 166s 3s/step - accuracy: 0.0764 - loss: 79.0411 - val_accuracy: 0.0058 - val_loss: 95.0361
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 156s 3s/step - accuracy: 0.0027 - loss: 78.4208 - val_accuracy: 0.0053 - val_loss: 96.2072
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 156s 3s/step - accuracy: 0.0033 - loss: 78.4125 - val_accuracy: 0.0071 - val_loss: 97.5718
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 155s 3s/step - accuracy: 0.0042 - loss: 78.4108 - val_accuracy: 0.0063 - val_loss: 98.8690
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 154s 3s/step - accuracy: 0.0039 - loss: 78.4098 - val_accuracy: 0.0053 - val_loss: 99.8585
Epoch 6/20
18/46 ━━━━━━━━━━━━━━━━━━━━ 1:30 3s/step - accuracy: 0.0024 - loss: 79.5490

KeyboardInterrupt: 